In [57]:
import torch
from torch.autograd import Variable
class MLPClassifier(torch.nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_heads):
        super(MLPClassifier, self).__init__()
        self.linear = torch.nn.Linear(input_dim, input_dim*num_heads)
        self.linear2 = torch.nn.Linear(input_dim*num_heads, input_dim*num_heads)
        self.linear3 = torch.nn.Linear(input_dim*num_heads, output_dim*num_heads)

        self.drop1 = torch.nn.Dropout(p=0.2)
        self.drop2 = torch.nn.Dropout(p=0.2)
        self.num_heads = num_heads
        self.output_dim = output_dim

    def forward(self, x):
        outputs = self.drop1(torch.nn.functional.relu(self.linear(x)))
        outputs = self.drop2(torch.nn.functional.relu(self.linear2(outputs)))
        outputs = self.linear3(outputs)
        return outputs.reshape(-1, self.num_heads, self.output_dim)

    def predict(self, encodings):
        vq_codes = []
        for X in encodings:
            inputs = X.cuda()

            outputs = model(inputs)[0]
        
            predicted = torch.argmax(torch.softmax(outputs, -1), dim=-1)

            vq_codes.append(predicted.cpu().tolist())
        return vq_codes

In [58]:
import json
from torchseq.agents.para_agent import ParaphraseAgent
from torchseq.datasets.json_loader import JsonDataLoader
from torchseq.utils.config import Config

import torch

path_to_model = '../models/separator-wa/'

examples = [
    {'input': 'What is the income for a soccer player?'},
    {'input': 'What do soccer players earn?'}
]

# First, get the encodings for the inputs

with open(path_to_model + "/config.json") as f:
    cfg_dict = json.load(f)
cfg_dict["env"]["data_path"] = "../data/"
cfg_dict["dataset"] = "json"
cfg_dict["json_dataset"] = {
    "path": None,
    "field_map": [
        {"type": "copy", "from": "input", "to": "s2"},
        {"type": "copy", "from": "input", "to": "template"},
        {"type": "copy", "from": "input", "to": "s1"},
    ],
}
cfg_dict["eval"]["sample_outputs"] = False
cfg_dict["bottleneck"]["prior_var_weight"] = 0.0

config = Config(cfg_dict)
    
data_loader = JsonDataLoader(config, test_samples=examples)

checkpoint_path = path_to_model + "/model/checkpoint.pt"

instance = ParaphraseAgent(config=config, run_id=None, output_path="./runs/parademo/", silent=True, verbose=False)

instance.load_checkpoint(checkpoint_path)
instance.model.eval()
    

_, _, _, mem = instance.inference(data_loader.test_loader, memory_keys_to_return=['sep_encoding_1','sep_encoding_2'])

del instance
encodings = torch.cat([mem['sep_encoding_1'], mem['sep_encoding_2']], dim=-1)

# Get MLP code predictions

NUM_HEADS = 16
NUM_TEMPL_HEADS = 4

input_dim = 768
output_dim = cfg_dict['bottleneck']['codebook_size']
hidden_dim = 768 *4

model = MLPClassifier(input_dim, output_dim, hidden_dim, NUM_TEMPL_HEADS).cuda()

model.load_state_dict(torch.load(path_to_model+'/code_predict.pt'))
model.eval()

vq_codes = model.predict(encodings)



# Pad the codes
vq_codes = [[0]*(NUM_HEADS-NUM_TEMPL_HEADS) + codes for codes in vq_codes]


# Now run generation
forced_examples = [{'vq_codes': codes, **x} for x, codes in zip(examples, vq_codes)]


cfg_dict["json_dataset"] = {
    "path": None,
    "field_map": [
        {"type": "copy", "from": "input", "to": "s2"},
        {"type": "copy", "from": "input", "to": "template"},
        {"type": "copy", "from": "input", "to": "s1"},
        {"type": "copy", "from": "vq_codes", "to": "forced_codes"},
    ],
}
cfg_dict["eval"]["sample_outputs"] = True
config = Config(cfg_dict)
data_loader2 = JsonDataLoader(config, test_samples=forced_examples)
instance2 = ParaphraseAgent(config=config, run_id=None, output_path="./runs/parademo/", silent=True, verbose=False)

instance2.load_checkpoint(checkpoint_path)
instance2.model.eval()

_, _, (pred_output, _, _), _ = instance2.inference(data_loader2.test_loader)

print(pred_output)

['how much is the salary of a soccer player?', 'how much do soccer players earn?']
